In [1]:
using Pkg, Knet, IterTools

In [2]:
# Convolutional Layer
struct Conv; w; b; f; end
(c::Conv)(x) = c.f.( pool( conv4(c.w, x).+ c.b ) )
Conv(w1,w2,cx,cy,f=relu) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f);

In [3]:
# Dense Layer
struct Dense; w; b; f; end
(d::Dense)(x) = d.f.( d.w * mat(x).+ d.b)
Dense(i::Int,o::Int,f=relu) = Dense(param(o,i), param0(o), f);

In [8]:
# Chain of layers
struct Chain; layers; Chain(args...) = new(args); end
(c::Chain)(x) = ( for l in c.layers; x = l(x); end; x )
(c::Chain)(x,y) = nll( c(x), y)

In [9]:
# load data
include(Knet.dir("data","mnist.jl"))
dtrn, dtst = mnistdata()

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Pkg\src\Pkg.jl:531


(Knet.Data{Tuple{Array{Float32,4},Array{UInt8,1}}}(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], UInt8[0x05 0x0a … 0x06 0x08], 100, 60000, false, 59901, 1:60000, false, (28, 28, 1, 60000), (60000,), Array{Float32,N} where N, Array{UInt8,1}), Knet.Data{Tuple{Array{Float32,4},Array{UInt8,1}}}(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], UInt8[0x07 0x02 … 0x05 0x06], 100, 10000, false, 9901, 1:10000, false, (28, 28, 1, 10000), (10000,), Array{Float32,N} where N, Array{UInt8,1}))

In [10]:
LeNet = Chain(Conv(5,5,1,20), Conv(5,5,20,50), Dense(800,500), Dense(500,10,identity))
progress!(adam(LeNet, ncycle(dtrn,10)))
accuracy(LeNet, dtst)

┣████████████████████┫ [100.00%, 6000/6000, 11:28/11:28, 8.72i/s]  ┫ [91.45%, 5487/6000, 10:22/11:20, 7.89i/s] 


0.9879